# Query 3

In [1]:
from sedona.spark import *
from pyspark.sql.functions import col, sum, collect_list, regexp_replace
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, count, avg
import time

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1599,application_1732639283265_1560,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def measure_join_performance(df, description):
    start_time = time.time()
    df.count()  # Trigger the join
    end_time = time.time()
    print(f"{description} took {end_time - start_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Query 3") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)

# Load the crime data, LA income and RE codes csv 
crime_data = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
la_income = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv", header=True, inferSchema=True)
police_stations = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv", header=True, inferSchema=True)
explanation = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks_fields.csv", header=True, inferSchema=True)


geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

# Formatting magic
flattened_blocks_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(flattened_blocks_df)
print(crime_data)
print(la_income)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[BG10: string, BG10FIP10: string, BG12: string, CB10: string, CEN_FIP13: string, CITY: string, CITYCOM: string, COMM: string, CT10: string, CT12: string, CTCB10: string, HD_2012: bigint, HD_NAME: string, HOUSING10: bigint, LA_FIP10: string, OBJECTID: bigint, POP_2010: bigint, PUMA10: string, SPA_2012: bigint, SPA_NAME: string, SUP_DIST: string, SUP_LABEL: string, ShapeSTArea: double, ShapeSTLength: double, ZCTA10: string, geometry: udt]
DataFrame[DR_NO: int, Date Rptd: string, DATE OCC: string, TIME OCC: int, AREA : int, AREA NAME: string, Rpt Dist No: int, Part 1-2: int, Crm Cd: int, Crm Cd Desc: string, Mocodes: string, Vict Age: int, Vict Sex: string, Vict Descent: string, Premis Cd: int, Premis Desc: string, Weapon Used Cd: int, Weapon Desc: string, Status: string, Status Desc: string, Crm Cd 1: int, Crm Cd 2: int, Crm Cd 3: int, Crm Cd 4: int, LOCATION: string, Cross Street: string, LAT: double, LON: double]
DataFrame[Zip Code: int, Community: string, Estimated Median Inc

In [5]:
explanation.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+-------------------------------------------------------------------------+
|field    |type  |meaning                                                                  |
+---------+------+-------------------------------------------------------------------------+
|BG10     |string|7-digit block group number (2010)                                        |
|BG10FIP10|string|Combination of BG10 and LA_FIP10 (Los Angeles County FIP code)           |
|BG12     |string|7-digit block group number (2012)                                        |
|CB10     |string|4-digit census block number                                              |
|CEN_FIP13|string|-                                                                        |
|CITY     |string|Incorporated city name                                                   |
|CITYCOM  |string|City/Community name label                                                |
|COMM     |string|Unincorporated area community name and LA City neigh

## Calculate population
The requested population per region is the total population of each region, etc Community. Below, we sum the total population of each region, and place it 
in the column "total_population"

In [9]:
flattened_blocks_df.select("COMM", "ZCTA10", "POP_2010", "HOUSING10").show()
populated_blocks = flattened_blocks_df.filter((flattened_blocks_df.POP_2010 !=0) & (flattened_blocks_df.HOUSING10 != 0))

populated_blocks.select("COMM", "ZCTA10", "POP_2010", "HOUSING10").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+--------+---------+
|                COMM|ZCTA10|POP_2010|HOUSING10|
+--------------------+------+--------+---------+
|West Antelope Valley| 93243|       0|        0|
|West Antelope Valley| 93243|       0|        0|
|West Antelope Valley| 93243|       0|        0|
|       South Edwards|      |       0|        0|
|       South Edwards|      |       0|        0|
|West Antelope Valley| 93243|       5|        5|
|       South Edwards|      |       0|        0|
|           San Pedro| 90732|      69|       26|
|         Diamond Bar| 91789|      79|       26|
| Rancho Palos Verdes| 90275|       0|        0|
|           San Pedro| 90731|     120|       70|
|           San Pedro| 90731|     240|       86|
|           San Pedro| 90731|       0|        0|
|           San Pedro| 90731|       0|        0|
|           San Pedro| 90732|      75|       29|
|           San Pedro| 90731|     246|       80|
|           San Pedro| 90731|     180|       87|
|           San Pedr

In [6]:
# Calculate total population per region (e.g., "Community", "COMM")
comm_pop = flattened_blocks_df.groupBy("COMM", "ZCTA10").agg(
    sum("POP_2010").alias("total_population"),
    sum("HOUSING10").alias("total_households")
)
comm_pop.select("COMM", "ZCTA10", "total_population", "total_households").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+----------------+----------------+
|                COMM|ZCTA10|total_population|total_households|
+--------------------+------+----------------+----------------+
|         West Vernon| 90037|           30161|            8556|
|            Glendale| 91204|           16032|            5969|
|                    | 91768|               0|               0|
|               Palms| 90232|            1533|             918|
|     Huntington Park| 90255|           57913|           15093|
|          Silverlake| 90039|           10522|            4757|
|          San Marino| 91108|           13147|            4477|
|            Westwood| 90025|            2443|            1317|
|Palos Verdes Estates| 90274|           13438|            5283|
|       South Carthay| 90035|            9428|            5250|
|                    | 90291|               0|               0|
|    Hawaiian Gardens| 90716|           14184|            3652|
|        Santa Monica| 90404|           

## Find the duplicates
Below is a code cell used to figure out the number of duplicate entries under the name "Community" in the la_income and the comm_pop data frames, with different zip codes and median incomes, as well as all the distinct "COMM" entries in the above community population data frame.
It is immediately evident, that the la_income contains several duplicates, whereas the comm_pop does not.

In [8]:
# Group by 'Community', count duplicates, and collect 'Estimated Median Income'
la_income.groupBy("Community").agg(
    collect_list("Zip Code").alias("zip"),
    count("*").alias("count"),
    collect_list("Estimated Median Income").alias("median_incomes")
).filter("count > 1").show(truncate=False)

comm_pop.groupBy("COMM").agg(
    count("*").alias("count")
).filter("count > 1").show(truncate=False)

comm_pop.select("COMM").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------+------------------------------------------+-----+-------------------------------------------------------+
|Community                 |zip                                       |count|median_incomes                                         |
+--------------------------+------------------------------------------+-----+-------------------------------------------------------+
|Alhambra                  |[91801, 91803]                            |2    |[$51,793, $56,928]                                     |
|Arcadia                   |[91006, 91007]                            |2    |[$83,986, $71,557]                                     |
|Beverly Hills             |[90210, 90211, 90212]                     |3    |[$145,227, $69,638, $92,476]                           |
|Burbank                   |[91501, 91502, 91505, 91506]              |4    |[$60,158, $40,057, $76,357, $71,495]                   |
|Carson                    |[90745, 90746]                    

## Average median income per Community 
In order to calculate the average median income out of all the Community entries in the la income data frame, we first clean the column from the $ characters. 
Then, we group the data frame by "Community" column, and calculate the average out of all the entries' "Estimated Median Income" columns

In [22]:
# Clean the 'Estimated Median Income' column from $ characters
la_income_clean = la_income.withColumn(
    "Estimated Median Income",
    regexp_replace(col("Estimated Median Income"), "[^0-9]", "").cast("double")
)

# Calculate average median income for each household in each community in the la_income csv
#avg_comm_income = la_income_clean.groupBy("Community").agg(
#    avg("Estimated Median Income").alias("average_median_income")
#)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
avg_comm_income.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+
|           Community|average_median_income|
+--------------------+---------------------+
|Hawthorne (Holly ...|              46172.0|
|         Culver City|              75913.5|
|Los Angeles (Down...|              22754.0|
|Los Angeles (Holl...|              57941.0|
| Los Angeles (Palms)|              58004.0|
|         Pico Rivera|              55758.0|
|Los Angeles (Hanc...|              40612.0|
|              Malibu|             123681.0|
|    Hacienda Heights|              78000.0|
|          Montebello|              45898.0|
|  Burbank (Glenoaks)|              70608.0|
|    Hawaiian Gardens|              37543.0|
|    Westlake Village|             104382.5|
|Los Angeles (Bald...|              36564.0|
|Glendale, La Cres...|              87612.0|
|              Carson|              74351.5|
|Los Angeles (Nort...|              42254.0|
|Los Angeles (Play...|              82055.0|
|Los Angeles (Sout...|              17018.0|
|         

## Join the avg_income and comm_pop data frames
Join the data frames on the "Community" ("COMM") column, in order to maintain a dataframe containing both the total population and the average income of the Community.
After using the explain() function, we notice that the join technique used by the catalyst optimizer is "BroadcastHashJoin".

In [11]:
comm_income = populated_blocks.join(
    la_income_clean, 
    ((populated_blocks["COMM"] == la_income_clean["Community"]) && ["Zip Code"] == la_income_clean[""), "inner"
)
comm_income.explain(True)  # Replace df with your DataFrame containing the join

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
Join Inner, (COMM#201 = Community#117)
:- Aggregate [COMM#201], [COMM#201, sum(POP_2010#210L) AS total_population#387L, sum(HOUSING10#207L) AS total_households#389L]
:  +- Project [properties#189.BG10 AS BG10#194, properties#189.BG10FIP10 AS BG10FIP10#195, properties#189.BG12 AS BG12#196, properties#189.CB10 AS CB10#197, properties#189.CEN_FIP13 AS CEN_FIP13#198, properties#189.CITY AS CITY#199, properties#189.CITYCOM AS CITYCOM#200, properties#189.COMM AS COMM#201, properties#189.CT10 AS CT10#202, properties#189.CT12 AS CT12#203, properties#189.CTCB10 AS CTCB10#204, properties#189.HD_2012 AS HD_2012#205L, properties#189.HD_NAME AS HD_NAME#206, properties#189.HOUSING10 AS HOUSING10#207L, properties#189.LA_FIP10 AS LA_FIP10#208, properties#189.OBJECTID AS OBJECTID#209L, properties#189.POP_2010 AS POP_2010#210L, properties#189.PUMA10 AS PUMA10#211, properties#189.SPA_2012 AS SPA_2012#212L, properties#189.SPA_NAME AS SPA_NAME#213, properties#189.SUP_DIST AS SUP_D

## Experiment with join strategies 
Force different join strategies using the hint() function, in order to test their efficiency. Apply a function that tests the time it takes for each join to complete and compare. 

In [23]:
comm_income_broadcast = comm_pop.join(
    avg_comm_income.hint("broadcast"),
    comm_pop["COMM"] == avg_comm_income["Community"]
)

comm_income_merge = comm_pop.join(
    avg_comm_income.hint("merge"),
    comm_pop["COMM"] == avg_comm_income["Community"]
)

comm_income_shuffle_hash = comm_pop.join(
    avg_comm_income.hint("shuffle_hash"),
    comm_pop["COMM"] == avg_comm_income["Community"]
).hint("shuffle_replicate_nl")

comm_income_shuffle_replicate_nl = comm_pop.join(
    avg_comm_income.hint("shuffle_replicate_nl"),
    comm_pop["COMM"] == avg_comm_income["Community"]
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
measure_join_performance(comm_income_broadcast, "Broadcast Join")
measure_join_performance(comm_income_merge, "Sort-Merge Join")
measure_join_performance(comm_income_shuffle_hash, "Shuffle Hash Join")
measure_join_performance(comm_income_shuffle_replicate_nl, "Replicated Nested Loop Join")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Broadcast Join took 20.43 seconds
Sort-Merge Join took 11.34 seconds
Shuffle Hash Join took 10.69 seconds
Replicated Nested Loop Join took 11.04 seconds

## Clean the dataframe
For a clean dataframe, keep only the columns of interest (etc "region", "total_population", "median_income").

In [12]:
comm_income = comm_income.select(
    comm_pop["COMM"].alias("region"), 
    "total_population", 
    col("average_median_income").alias("median_income"),
    "total_households"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Calculate income per person 
Multiply the number of households with the household median income, and divide the result with the total population of each region.

In [13]:
# Calculate income per person 
income_per_person = comm_income.withColumn(
    "income_per_person",
    col("total_households") * col("median_income") / col("total_population")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
income_per_person.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+------------------+----------------+------------------+
|              region|total_population|     median_income|total_households| income_per_person|
+--------------------+----------------+------------------+----------------+------------------+
|         Culver City|           38883|           75913.5|           17491|34148.677532597794|
|         Pico Rivera|           62942|           55758.0|           17109|15156.233071716819|
|              Malibu|           12645|          123681.0|            6864| 67136.92241992883|
|    Hacienda Heights|           53594|           78000.0|           16524| 24048.81143411576|
|          Montebello|           62500|           45898.0|           19768|      14516.986624|
|    Hawaiian Gardens|           14254|           37543.0|            3703| 9753.173074224778|
|    Westlake Village|            8270|          104382.5|            3384| 42712.25876662636|
|              Carson|           91714|           

## Crime data Geometry join
Add the geometry column in the crime_data and join the dataframe with the census blocks dataframe to combine the crime data with their location. 
After using the explain() function, we notice that the join technique used by the catalyst optimizer is "RangeJoin".

In [15]:
# Create geometry column for crime data
crime_data = crime_data.withColumn("geom", ST_Point(col("LON"), col("LAT")))
crime_data_location = crime_data.join(flattened_blocks_df, ST_Within(crime_data["geom"], flattened_blocks_df["geometry"]), "inner")
crime_data_location.explain(True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
Join Inner,  **org.apache.spark.sql.sedona_sql.expressions.ST_Within**
:- Project [DR_NO#42, Date Rptd#43, DATE OCC#44, TIME OCC#45, AREA #46, AREA NAME#47, Rpt Dist No#48, Part 1-2#49, Crm Cd#50, Crm Cd Desc#51, Mocodes#52, Vict Age#53, Vict Sex#54, Vict Descent#55, Premis Cd#56, Premis Desc#57, Weapon Used Cd#58, Weapon Desc#59, Status#60, Status Desc#61, Crm Cd 1#62, Crm Cd 2#63, Crm Cd 3#64, Crm Cd 4#65, ... 5 more fields]
:  +- Relation [DR_NO#42,Date Rptd#43,DATE OCC#44,TIME OCC#45,AREA #46,AREA NAME#47,Rpt Dist No#48,Part 1-2#49,Crm Cd#50,Crm Cd Desc#51,Mocodes#52,Vict Age#53,Vict Sex#54,Vict Descent#55,Premis Cd#56,Premis Desc#57,Weapon Used Cd#58,Weapon Desc#59,Status#60,Status Desc#61,Crm Cd 1#62,Crm Cd 2#63,Crm Cd 3#64,Crm Cd 4#65,... 4 more fields] csv
+- Project [properties#189.BG10 AS BG10#194, properties#189.BG10FIP10 AS BG10FIP10#195, properties#189.BG12 AS BG12#196, properties#189.CB10 AS CB10#197, properties#189.CEN_FIP13 AS CEN_FIP13#198, pr

## Group by Community and count the number of crimes
Group the crime_pop dataframe by "COMM" column, and count the number of crimes commited per region.

In [16]:
crime_pop = crime_data_location.groupBy("COMM").agg(
    count("*").alias("total_crime_number")
)
crime_pop.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|                COMM|total_crime_number|
+--------------------+------------------+
|    Westlake Village|                 1|
|        Agoura Hills|                 6|
|        Hidden Hills|                 3|
|Santa Monica Moun...|                 4|
|         Canoga Park|             37017|
|            Winnetka|             22947|
|           Westhills|                 7|
|          West Hills|             13065|
|           Calabasas|                11|
|      Woodland Hills|             31767|
|                    |                37|
|      West Hollywood|               495|
|           Thai Town|              3124|
|           Hollywood|             63825|
|      Little Armenia|              4192|
|             Melrose|             43800|
|      Gramercy Place|              7947|
| Rancho Palos Verdes|               131|
|             Gardena|               353|
|        Harbor Pines|               772|
+--------------------+------------

## Join the income_per_person and crime_pop dataframes
Join the two dataframes on the "region" ("COMM") columns, to combine all the income, crime and population data in a single dataframe. Then, add the "crimes_per_person" column, dividingg the total crime number of each region with its total population. After using the explain() function, we notice that the join technique used by the catalyst optimizer is "SortMergeJoin".

In [17]:
# Join income and population data with crime data
result = income_per_person.join(
    crime_pop,
    income_per_person["region"] == crime_pop["COMM"]
)
result.explain(True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
Join Inner, (region#569 = COMM#201)
:- Project [region#569, total_population#387L, median_income#570, total_households#389L, ((cast(total_households#389L as double) * median_income#570) / cast(total_population#387L as double)) AS income_per_person#575]
:  +- Project [COMM#201 AS region#569, total_population#387L, average_median_income#518 AS median_income#570, total_households#389L]
:     +- Join Inner, (COMM#201 = Community#117)
:        :- Aggregate [COMM#201], [COMM#201, sum(POP_2010#210L) AS total_population#387L, sum(HOUSING10#207L) AS total_households#389L]
:        :  +- Project [properties#189.BG10 AS BG10#194, properties#189.BG10FIP10 AS BG10FIP10#195, properties#189.BG12 AS BG12#196, properties#189.CB10 AS CB10#197, properties#189.CEN_FIP13 AS CEN_FIP13#198, properties#189.CITY AS CITY#199, properties#189.CITYCOM AS CITYCOM#200, properties#189.COMM AS COMM#201, properties#189.CT10 AS CT10#202, properties#189.CT12 AS CT12#203, properties#189.CTCB10 AS

## Experiment with join strategies 
Force different join strategies using the hint() function, in order to test their efficiency. Apply a function that tests the time it takes for each join to complete and compare. 

In [25]:
# Example join DataFrames
result_broadcast = income_per_person.join(
    crime_pop.hint("broadcast"),
    income_per_person["region"] == crime_pop["COMM"]
)

result_merge = income_per_person.join(
    crime_pop.hint("merge"),
    income_per_person["region"] == crime_pop["COMM"]
)

result_shuffle_hash = income_per_person.join(
    crime_pop.hint("shuffle_hash"),
    income_per_person["region"] == crime_pop["COMM"]
)

result_shuffle_replicate_nl = income_per_person.join(
    crime_pop.hint("shuffle_replicate_nl"),
    income_per_person["region"] == crime_pop["COMM"]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
measure_join_performance(result_broadcast, "Broadcast Join")
measure_join_performance(result_merge, "Sort-Merge Join")
measure_join_performance(result_shuffle_hash, "Shuffle Hash Join")
measure_join_performance(result_shuffle_replicate_nl, "Replicated Nested Loop Join")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Broadcast Join took 41.81 seconds
Sort-Merge Join took 24.93 seconds
Shuffle Hash Join took 23.65 seconds
Replicated Nested Loop Join took 19.37 seconds

In [ ]:
result = result.withColumn(
    "crimes_per_person",
    col("total_crime_number") / col("total_population")
)

In [66]:
result.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+------------------+----------------+------------------+--------------------+------------------+--------------------+
|              region|total_population|     median_income|total_households| income_per_person|                COMM|total_crime_number|   crimes_per_person|
+--------------------+----------------+------------------+----------------+------------------+--------------------+------------------+--------------------+
|         Culver City|           38883|           75913.5|           17491|34148.677532597794|         Culver City|              1014|0.026078234704112337|
|         Pico Rivera|           62942|           55758.0|           17109|15156.233071716819|         Pico Rivera|                 2|3.177528518318452E-5|
|              Malibu|           12645|          123681.0|            6864| 67136.92241992883|              Malibu|                 1|7.908264136022143E-5|
|          Montebello|           62500|           45898.0|      